In [1]:
# 텐서보드 적용 해봤습니다.
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-07-26 07:14:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.55.191.55, 35.172.177.65, 54.152.127.232, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.55.191.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  17.6MB/s    in 0.7s    

2019-07-26 07:14:12 (17.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://ad3b2d8f.ngrok.io


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.callbacks import TensorBoard

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1] * train_images.shape[2]).astype('float32') / 255
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1] * test_images.shape[2]).astype('float32') / 255

In [0]:
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)

In [6]:
#fully-connected 와 activation 사이에 batch normalization 부분을 추가하였습니다.
model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('softmax'))

W0726 07:14:18.968322 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 07:14:19.010554 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 07:14:19.020330 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 07:14:19.114412 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0726 07:14:19.226239 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 07:14:19.255758 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
tbCallBack = TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=512,
                         write_images=True)

In [9]:
model.fit(train_images, train_labels, epochs=300, verbose=1, validation_data=(test_images, test_labels), batch_size=512, callbacks=[tbCallBack])

W0726 07:14:21.865829 139710724269952 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples


W0726 07:14:25.501496 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:796: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0726 07:14:25.818604 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:840: The name tf.summary.image is deprecated. Please use tf.compat.v1.summary.image instead.

W0726 07:14:28.547562 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0726 07:14:28.551257 139710724269952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/300
60000/60000 [==============================] - 2s 35us/step - loss: 0.8015 - acc: 0.7999 - val_loss: 0.7199 - val_acc: 0.8310
Epoch 2/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.5815 - acc: 0.8619 - val_loss: 0.6061 - val_acc: 0.8506
Epoch 3/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.4993 - acc: 0.8762 - val_loss: 0.5454 - val_acc: 0.8617
Epoch 4/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.4455 - acc: 0.8854 - val_loss: 0.4951 - val_acc: 0.8670
Epoch 5/300
60000/60000 [==============================] - 1s 19us/step - loss: 0.4049 - acc: 0.8938 - val_loss: 0.5229 - val_acc: 0.8523
Epoch 6/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.3728 - acc: 0.8983 - val_loss: 0.4634 - val_acc: 0.8613
Epoch 7/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.3444 - acc: 0.9051 - val_loss: 0.4104 - val_acc: 0.8751
Epoch 8/300
60000/60000 [=========

In [10]:
print("Accuracy : %.4f" % (model.evaluate(test_images, test_labels)[1]))
# batch normalization의 경우, epoch을 늘리고 줄이고 해봤는데 원래 모델보다 accuracy가 향상되지가 않았습니다.. 

10000/10000 [==============================] - 1s 50us/step
Accuracy : 0.8687
